In [10]:
import cv2
import numpy as np
import face_recognition
from datetime import datetime
import pickle
import requests
import os

In [11]:

path = 'Training_images'
images = []
classNames = []
myList = os.listdir(path)
print(myList)
for cl in myList:
    curImg = cv2.imread(f'{path}/{cl}')
    images.append(curImg)
    classNames.append(os.path.splitext(cl)[0])
print(classNames)


def findEncodings(images):
    encodeList = []


    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encode = face_recognition.face_encodings(img)[0]
        encodeList.append(encode)
    return encodeList

['tushargarg.jpeg', 'tusharkumar.jpeg', 'yogii.jpeg']
['tushargarg', 'tusharkumar', 'yogii']


In [12]:
cap = cv2.VideoCapture(0)
mode = pickle.load(open("encodings.pkl", "rb"))
UNKNOWN_LABEL = "Unknown"
arr = []
print(f"no person is present till now")
classNames = ['tushargarg', 'tusharkumar', 'yogii']
while True:
    success, img = cap.read()
    imgS = cv2.resize(img, (0, 0), None, 0.25, 0.25)
    imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)

    facesCurFrame = face_recognition.face_locations(imgS)
    encodesCurFrame = face_recognition.face_encodings(imgS, facesCurFrame)

    for encodeFace, faceLoc in zip(encodesCurFrame, facesCurFrame):
        matches = face_recognition.compare_faces(mode, encodeFace)
        faceDis = face_recognition.face_distance(mode, encodeFace)
        matchIndex = np.argmin(faceDis)

        if matches[matchIndex]:
            name = classNames[matchIndex].upper()
            y1, x2, y2, x1 = faceLoc
            y1, x2, y2, x1 = y1 * 4, x2 * 4, y2 * 4, x1 * 4
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.rectangle(img, (x1, y2 - 35), (x2, y2), (0, 255, 0), 2)
            cv2.putText(img, name, (x1 + 6, y2 - 6), cv2.FONT_HERSHEY_DUPLEX, 1, (255, 255, 255), 2)
            if name in arr:
                continue
            else:
                arr.append(name)
                print(arr)
        else:
            # Face is not recognized, label it as "Unknown"
            y1, x2, y2, x1 = faceLoc
            y1, x2, y2, x1 = y1 * 4, x2 * 4, y2 * 4, x1 * 4
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 255), 2)  # Red border
            cv2.rectangle(img, (x1, y2 - 35), (x2, y2), (0, 0, 255), cv2.FILLED)
            cv2.putText(img, UNKNOWN_LABEL, (x1 + 6, y2 - 6), cv2.FONT_HERSHEY_DUPLEX, 1, (255, 255, 255), 2)
            name = 'Unknown'
            if name in arr:
                continue
            else:
                arr.append(name)
#                 print(arr)
            if(len(arr)>10):
                arr.pop(0)
        # Display the resulting frame
    cv2.imshow('Face Recognition', img)

    # Exit loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close OpenCV windows
cap.release()
cv2.destroyAllWindows()


no person is present till now
['YOGII']


# 